In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from statannotations.Annotator import Annotator


In [ ]:
# Load the CSV file
filepath = '...'
df = pd.read_csv(os.path.join(filepath, 'Directionality_quantification.csv'))

In [ ]:
cross_tab = pd.crosstab(df['PDMS'], df['Direction'])

# Calculate percentages for each PDMS type
percentages = cross_tab.div(cross_tab.sum(axis=1), axis=0) * 100

# Create a mapping for the full names
pdms_names = {
    'P': 'Plain',
    'H': 'Heart',
    'EP': 'En Passant'
}

# Set up the plot
plt.figure(figsize=(6, 7))

# Set the color palette - different shades of blue
blues = sns.color_palette("Blues", n_colors=2)

# Create the bar chart
bar_width = 0.35
index = np.arange(len(percentages.index))

# Create the bar chart
bars1 = plt.bar(index, percentages['F'], bar_width, color='dimgray', label='Forward')
bars2 = plt.bar(index, percentages['B'], bar_width, bottom=percentages['F'], 
                color='dimgray', alpha=0.5, label='Backward')
# Calculate total count for each PDMS type
totals = cross_tab.sum(axis=1)

# Create x-labels with total counts
x_labels = [f"{pdms_names[pdms]}\nN={totals[pdms]}" for pdms in percentages.index]

# Add labels without title
plt.xlabel('PDMS Type', fontsize=20)
plt.ylabel('Neurite Growth Direction [%]', fontsize=20)
plt.xticks(index, x_labels, fontsize=18)
plt.yticks(fontsize=18)
plt.grid(True, axis="y", linestyle=':', alpha=0.7)

plt.legend(fontsize=16)

# Add percentage labels on the bars
def add_labels(bars, percentages):
    for bar in bars:
        height = bar.get_height()
        if height > 0:  # Add label for all bars
            plt.text(bar.get_x() + bar.get_width()/2., bar.get_y() + height/2.,
                     f'{int(height)}%', ha='center', va='center', color='white', fontweight='bold', fontsize=12)

add_labels(bars1, percentages['F'])
add_labels(bars2, percentages['B'])

plt.ylim(0, 110)  # Extend y-axis to make room for the count labels
plt.tight_layout()
plt.savefig(os.path.join(filepath, 'directionality_percentages.png'), dpi=300)
plt.savefig(os.path.join(filepath, 'directionality_percentages.pdf'), dpi=300)
plt.show()


Giulia style plotting

In [ ]:
df = pd.read_csv(os.path.join(filepath, 'Filled_quantification.csv'))
# Calculate the percentage filled
df['Percentage_Filled'] = (df['Filled'] / df['Total'] * 100).round(1)

# Create a more descriptive experiment label
df['Experiment'] = df['PDMS'] + ' - Sample ' + df['Sample'].astype(str)


In [ ]:
def custom_boxplot(data, x_metric, y_metric, hue_metric, axes, labely, fontsize=16, labelsize=14, position_line=-0.12):
 
    # Plot the strip plot
    sns.boxplot(data=data, x=x_metric, y=y_metric, color = 'dimgray',
                 legend=False, zorder=1, ax=axes, width=0.4, fliersize=0, showfliers=False)
 
    # Add statistical annotations
    pairs = [
        ("H", "P"),
        ("EP", "H"),
        ("P", "EP"),
    ]
 
    print(data.columns)
    annotator = Annotator(axes, pairs, data=data, x=hue_metric, y=y_metric)
    annotator.configure(test='Kruskal', text_format='star', loc='inside', fontsize=fontsize, hide_non_significant=True)
    print(f"\nAnnotate for metric {y_metric}")
    annotator.apply_and_annotate()
    
    plt.rcParams['xtick.bottom'] = True
    plt.rcParams['ytick.left'] = True
    plt.rcParams['font.family'] = 'Arial'
    axes.set_ylabel(labely, fontsize=fontsize)
    axes.set_xlabel("", fontsize=fontsize)
    axes.grid(True, axis="y", linestyle=':', alpha=0.7)
    axes.tick_params(axis='both', which='major', labelsize=labelsize, width=1.5, length=8)
    #axes.yaxis.set_major_locator(ticker.MaxNLocator(5))
    
    # Add grouped x-axis labels
    axes.tick_params(axis='both', which='major')
    xticks = axes.get_xticks()  # Positions of "Soft", "Inter", "Stiff", etc.
    print(xticks)
    axes.set_xticks(xticks)  # Position of ticks
    axes.set_xticklabels(["En Passant", "Heart", "Straight"])
    
    # Get relative x-ticks position
    x_min, x_max = axes.get_xlim()
    #ticks = [(tick - x_min) / (x_max - x_min) for tick in axes.get_xticks()]
    plt.savefig(os.path.join(filepath, 'channel_filling_analysis_w_statistics.png'), dpi=300)
    plt.savefig(os.path.join(filepath, 'channel_filling_analysis_w_statistics.pdf'), dpi=300)
 
    


In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
custom_boxplot(df, 'PDMS', 'Percentage_Filled', 'PDMS', ax, labely='Percentage of Wells With Cells [%]')


In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
custom_boxplot(df, 'PDMS', 'Percentage_Filled', 'PDMS', ax, labely='Percentage of Wells With Cells [%]')


In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
df = pd.read_csv(os.path.join(filepath, 'Filled_quantification.csv'))
# Calculate the percentage filled
df['Percentage_Filled'] = (df['Filled'] / df['Total']).round(1)

# Create a more descriptive experiment label
df['Experiment'] = df['PDMS'] + ' - Sample ' + df['Sample'].astype(str)

In [ ]:
def custom_boxplot_binomial(data, x_metric, y_metric, hue_metric, axes, labely, fontsize=20, labelsize=18, position_line=-0.12):
    import numpy as np
    import seaborn as sns
    import matplotlib.pyplot as plt
    import os
    #remove warnings
    import warnings
    warnings.filterwarnings("ignore", category=UserWarning, module='seaborn')
    
    # Define the mapping from dataframe values to display labels
    label_mapping = {
        "P": "Straight",
        "H": "Heart", 
        "EP": "En-Passant"
    }
    
    # Set the desired order for x-axis categories (in terms of dataframe values)
    data_order = ["P", "H", "EP"]  # This corresponds to Straight, Heart, En Passant
    
    # Plot the boxplot with the data order
    sns.boxplot(data=data, x=x_metric, y=y_metric, color='dimgray',
                order=data_order, legend=False, zorder=1, ax=axes, width=0.4, 
                fliersize=0, showfliers=False)
    
    # Calculate binomial confidence intervals for each category
    binomial_errors = {}
    for category in data_order:
        category_data = data[data[x_metric] == category][y_metric]
        n = len(category_data)
        p = category_data.mean()  # proportion/mean

        print(n,p)
        
        # Calculate binomial standard error: sqrt(p(1-p)/n)
        if n > 0 and 0 < p < 1:
            error = 100 * np.sqrt((p * (1 - p)) / n)
        else:
            error = 0
        
        binomial_errors[category] = {
            'mean': p,
            'error': error,
            'n': n
        }
        
        print(f"{label_mapping[category]}: mean={p:.3f}, error=±{error:.1f}%, n={n}")
    
    # Add error bars to the boxplot
    x_positions = range(len(data_order))
    for i, category in enumerate(data_order):
        mean_val = binomial_errors[category]['mean']
        error_val = binomial_errors[category]['error'] / 100  # Convert back to proportion
        
        # Add error bars
        axes.errorbar(i, mean_val, yerr=error_val, 
                     fmt='o', color='red', markersize=6, 
                     capsize=5, capthick=2, elinewidth=2,
                     zorder=10, label='Binomial CI' if i == 0 else "")
    
    # Add text annotations showing the error percentages
    for i, category in enumerate(data_order):
        error_pct = binomial_errors[category]['error']
        mean_val = binomial_errors[category]['mean']
        axes.text(i, mean_val + 0.05, f'±{error_pct:.1f}%', 
                 ha='center', va='bottom', fontsize=fontsize-4, 
                 bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))
    
    plt.rcParams['xtick.bottom'] = True
    plt.rcParams['ytick.left'] = True
    axes.set_ylabel(labely, fontsize=fontsize)
    axes.set_xlabel("", fontsize=fontsize)
    axes.grid(True, axis="y", linestyle=':', alpha=0.7)
    axes.tick_params(axis='both', which='major', labelsize=labelsize, width=1.5, length=8)
    
    # Set the display labels based on the data order
    display_labels = [label_mapping[cat] for cat in data_order]
    
    # Update x-tick labels with the mapped display names
    axes.tick_params(axis='both', which='major')
    xticks = axes.get_xticks()
    axes.set_xticks(xticks)
    axes.set_xticklabels(display_labels)
    
    # Add legend for error bars
    axes.legend(loc='upper right', fontsize=labelsize-2)
    
    plt.savefig(os.path.join(filepath, 'channel_filling_analysis_w_binomial_errors.png'), dpi=300)
    plt.savefig(os.path.join(filepath, 'channel_filling_analysis_w_binomial_errors.pdf'), dpi=300)

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
custom_boxplot_binomial(df, 'PDMS', 'Percentage_Filled', 'PDMS', ax, labely='Percentage of Wells With Cells [%]')
